In [ ]:
# from google.colab import files #one way to import files
# uploaded = files.upload()

In [ ]:
# import os
# os.listdir()
# !pwd #check working directory

In [ ]:
import torch
from torch import nn
import sklearn

device = "cuda" if torch.cuda.is_available() else "cpu" # device agnostic code

In [ ]:
import pandas as pd
import numpy as py

inputs_dataframe = pd.read_csv('full.whet.multimer.csv') #read from working directory
inputs_dataframe.drop(inputs_dataframe.columns[[0, 13, 14, 15, 16]], axis=1, inplace=True) #delete variant column and the binary columns at the end
outputs_dataframe = pd.read_csv('a1q1.model_data.csv')
outputs_dataframe.drop(outputs_dataframe.columns[[0]], axis=1, inplace=True) #delete variant column

inputs_dataframe.fillna(0, inplace=True) #replace NaNs with zeros in the input data

print(outputs_dataframe) # visualize data before NaN deletion

outputs_dataframe.fillna(0, inplace=True) #replace NaNs with zeros in the target data

print(outputs_dataframe) # visualize data after NaN deletion

inputs_py = inputs_dataframe.to_numpy() #numpy array from dataframe
outputs_py = outputs_dataframe.to_numpy()

inputs = torch.from_numpy(inputs_py).type(torch.float) #create tensor from numpy array
outputs = torch.from_numpy(outputs_py).type(torch.float)

print(f"Imported input dataset of shape: {inputs.shape}") #confirm tensor shape
print(f"Imported output dataset of shape: {outputs.shape}")

#print(f"First 20 input rows: {inputs[:20]}")
#print(f"First 20 output rows: {inputs[:20]}")

     Peak Current Density  V_1/2  tau_act  tau_deac
0                    1.00  1.000     1.00      1.00
1                    1.00  1.000     1.00      1.00
2                    1.00  1.000     1.00      1.00
3                    1.00  1.000     1.00      1.00
4                    1.00  1.000     1.00      1.00
..                    ...    ...      ...       ...
465                  1.59  1.050     1.28      0.99
466                  0.22  1.741     1.54       NaN
467                  1.00  1.000     1.00      1.00
468                  1.00  1.000     1.00      1.00
469                  1.00  1.000     1.00      1.00

[470 rows x 4 columns]
     Peak Current Density  V_1/2  tau_act  tau_deac
0                    1.00  1.000     1.00      1.00
1                    1.00  1.000     1.00      1.00
2                    1.00  1.000     1.00      1.00
3                    1.00  1.000     1.00      1.00
4                    1.00  1.000     1.00      1.00
..                    ...    ...      ..

How to handle NaNs??????

In [ ]:
from pathlib import Path

MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True) # make model directory to save models to

class Model(nn.Module): #create NN class
  def __init__(self, input_features, output_features, hidden_units):
    super().__init__()

    self.layer_stack = nn.Sequential(
        nn.Linear(in_features=input_features, out_features=hidden_units),
        nn.LeakyReLU(),
        nn.Dropout(0.25),
        nn.Linear(in_features=hidden_units, out_features=hidden_units),
        nn.LeakyReLU(),
        nn.Dropout(0.25),
        nn.Linear(in_features=hidden_units, out_features=hidden_units),
        nn.LeakyReLU(),
        nn.Linear(in_features=hidden_units, out_features=output_features)
    )
        
  def forward(self, x):
    return self.layer_stack(x)

KCNQ=Model(input_features=inputs.size(1), output_features=outputs.size(1), hidden_units=32).to(device) #Create the model using 32 units in both hidden layers
KCNQ5=Model(input_features=inputs.size(1), output_features=outputs.size(1), hidden_units=32).to(device) #Create a model named KCNQ5 which was the intended averaged model of the
# k-fold cross validation as the averaged parameters of k=0 ... k=3 would be averaged for the k=4 fold of the dataset. The models whose parameters would be averaged
# would be saved as KCNQ1 through KCNQ4 and therefore the averaged one would be KCNQ5

print("The parameters: ", list(KCNQ.parameters())) #print starting parameters 

MODEL_NAME = "KCNQStarter.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

print(f"Saving model to: {MODEL_SAVE_PATH}") #save the starting parameters to reset with each interation of k through the cross validation
torch.save(obj=KCNQ.state_dict(), f=MODEL_SAVE_PATH)

RuntimeError: ignored

In [ ]:
# class AltModel(nn.Module): #another way to define the NN
#   def __init__(self, input_features, output_features, hidden_units):
#     super().__init__()
#     self.layer_1 = nn.Linear(in_features=input_features, out_features=hidden_units)
#     self.layer_2 = nn.Linear(in_features=hidden_units, out_features=hidden_units)
#     self.layer_3 = nn.Linear(in_features=hidden_units, out_features=output_features)
#     self.relu = nn.LeakyReLU()

#   def forward(self, x):
#     return self.layer_3(self.relu(self.layer_2(self.relu(self.layer_1(x)))))

# KCNQ_Model2 = AltModel(input_features=inputs.size(1), output_features=outputs.size(1), hidden_units=inputs.size(1)).to(device)

# # print("The parameters: ", list(KCNQ_Model2.parameters()))  

In [ ]:
#loss_fn = nn.MSELoss()
loss_fn = nn.SmoothL1Loss() # selecting the loss function
optimizer = torch.optim.SGD(params=KCNQ.parameters(), lr=0.01) # selecting the optimizer

In [ ]:
# train test split of the data as 80/20 for my first attempt at training this model

# from sklearn.model_selection import train_test_split

# input_train, input_test, output_train, output_test = train_test_split(inputs, outputs, test_size=0.2, random_state=42)

In [ ]:
# code to train the model as a 80/20 train test split

# torch.manual_seed(42)

# epochs = 10000

# epoch_count = [] #empty tensors that can be appended as you run through the epochs
# loss_values = []
# test_loss_values = []

# input_train, output_train = input_train.to(device), output_train.to(device) #make sure the data are on the selected device (either GPU or CPU)
# input_test, output_test = input_test.to(device), output_test.to(device)

# for epoch in range(epochs):
#   KCNQ.train()

#   output_preds = KCNQ(input_train) #generate output preditions
  
#   loss = loss_fn(output_preds, output_train) #calculate loss
#   loss_values.append(loss.item()) #save the loss of this epoch

#   optimizer.zero_grad() #zero the optimizer

#   loss.backward() #backpropagation

#   optimizer.step() #gradient descent

#   KCNQ.eval() #set the model to eval mode



#   with torch.inference_mode():
#     test_preds = KCNQ(input_test) #generate test prediction with this epoch's model parameters
#     test_loss = loss_fn(test_preds, output_test) #calculate test loss with this epoch's model parameters
#     epoch_count.append(epoch) #save the epoch count
#     #loss_values.append(loss) #save the training loss, commented out because it was moved above
#     test_loss_values.append(test_loss) # save the test loss

#   if epoch % 1000 == 0:
    
#     print(f"Epoch: {epoch} | Loss: {loss:.4f} | Test loss: {test_loss:.4f}") 
  

In [ ]:
# plot the training and test losses as a function of epoch

# import numpy as np
# import matplotlib.pyplot as plt

# plt.plot(epoch_count, np.array(torch.tensor(loss_values).numpy()), label="Train loss")
# plt.plot(epoch_count, torch.Tensor(test_loss_values), label="Test loss")
# plt.title("Training and test loss curves")
# plt.xlabel("Epochs")
# plt.ylabel("Normalized Loss")
# plt.legend();


In [ ]:
# just visualizing the test predictions versus the actual test data

# with torch.inference_mode():
#   test_preds = KCNQ(input_test)
# test_preds[:10], output_test[:10]

In [ ]:
# similar comparison

# a = output_preds[295:300]
# b = output_train[295:300]
# print(a)
# print(b)
# loss = loss_fn(a, b)
# print(loss)

In [ ]:
# implementing the model using k-fold cross validation

import numpy as np
from sklearn.model_selection import KFold
from pathlib import Path


MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)
LOAD_MODEL_NAME = "KCNQStarter.pth" #above the starting random model parameters were saved as this

MODEL_LOAD_PATH = MODEL_PATH / LOAD_MODEL_NAME

torch.manual_seed(42)

epochs = 10000

n_splits = 5

epoch_count = torch.zeros(epochs,1) #setting up tensors of appropriate size based on the number of splits (k)
loss_values = torch.zeros(epochs,n_splits-1)
test_loss_values = torch.zeros(epochs,n_splits-1)

input_train_splits = torch.zeros(int((inputs.size(0)*(n_splits-1))/n_splits), inputs.size(1), n_splits) #setting up tensors of appropriate size based on the number of splits (k)
input_test_splits = torch.zeros(int(inputs.size(0)/n_splits), inputs.size(1), n_splits) 

output_train_splits = torch.zeros(int((outputs.size(0)*(n_splits-1))/n_splits), outputs.size(1), n_splits) #setting up tensors of appropriate size based on the number of splits (k)
output_test_splits = torch.zeros(int(outputs.size(0)/n_splits), outputs.size(1), n_splits)

#print(input_train_splits.shape)
#print(input_test_splits.shape)

kf = KFold(n_splits, random_state=42, shuffle=True)
kf.get_n_splits(inputs)

# print(kf)

for i, (train_index, test_index) in enumerate(kf.split(inputs)): #theoretically this should be retrieving every train and test index in each split as specified by KFold, i think this might be where the issue is
  #print(f"Fold {i}:")
  #print(f"  Train: index={train_index}")
  #print(f"  Test:  index={test_index}")
  
  input_train_splits[:,:,i] = torch.index_select(inputs, 0, torch.from_numpy(train_index)) # this should be assigning all of the input train and test splits the values from the appropriate indices
  #print(train_splits[:,:,i])
  input_test_splits[:,:,i] = torch.index_select(inputs, 0, torch.from_numpy(test_index))
  output_train_splits[:,:,i] = torch.index_select(outputs, 0, torch.from_numpy(train_index))
  output_test_splits[:,:,i] = torch.index_select(outputs, 0, torch.from_numpy(test_index))

  if i != 4:
    for epoch in range(epochs):
      #KCNQ.load_state_dict(torch.load(f=MODEL_LOAD_PATH))
      KCNQ.train()

      output_preds = KCNQ(input_train_splits[:,:,i]) #similar implementation as the 80/20 train test split model as above
      
      loss = loss_fn(output_preds, output_train_splits[:,:,i])
      loss_values[epoch,i-1] = loss

      optimizer.zero_grad()

      loss.backward() #backpropagation

      optimizer.step() #gradient descent

      KCNQ.eval()

      with torch.inference_mode():
        test_preds = KCNQ(input_test_splits[:,:,i])
        test_loss = loss_fn(test_preds, output_test_splits[:,:,i])
        test_loss_values[epoch,i] = test_loss

      if epoch % 2000 == 0:
        print(f"Epoch: {epoch} | Loss: {loss:.4f} | Test loss: {test_loss:.4f}")

      if epoch % 9999 == 0 and epoch != 0 : # this was my attempt to save the parameters of each model on the i=9999 (10000th) epoch as a separate model
        MODEL_NAME = str.join('.', ('KCNQ_Model', str(i), 'pth'))
        MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

        #print(f"Saving model to: {MODEL_SAVE_PATH}")
        torch.save(obj=KCNQ.state_dict(), f=MODEL_SAVE_PATH)

        KCNQ.load_state_dict(torch.load(f=MODEL_LOAD_PATH))
  else:
    print("hi")
    # KCNQ1 = Model(input_features=inputs.size(1), output_features=outputs.size(1), hidden_units=inputs.size(1)).to(device) # this was my attempt to load the parameters of each model as above and then average them into one model termed KCNQ5 
    # MODEL_SAVE_PATH = MODEL_PATH / 'KCNQ_Model.0.pth'
    # KCNQ1.load_state_dict(torch.load(f=MODEL_SAVE_PATH))
    # KCNQ1.to(device)
    # KCNQ2 = Model(input_features=inputs.size(1), output_features=outputs.size(1), hidden_units=inputs.size(1)).to(device)
    # MODEL_SAVE_PATH = MODEL_PATH / 'KCNQ_Model.1.pth'
    # KCNQ2.load_state_dict(torch.load(f=MODEL_SAVE_PATH))
    # KCNQ2.to(device)
    # KCNQ3 = Model(input_features=inputs.size(1), output_features=outputs.size(1), hidden_units=inputs.size(1)).to(device)
    # MODEL_SAVE_PATH = MODEL_PATH / 'KCNQ_Model.2.pth'
    # KCNQ3.load_state_dict(torch.load(f=MODEL_SAVE_PATH))
    # KCNQ3.to(device)
    # KCNQ4 = Model(input_features=inputs.size(1), output_features=outputs.size(1), hidden_units=inputs.size(1)).to(device)
    # MODEL_SAVE_PATH = MODEL_PATH / 'KCNQ_Model.3.pth'
    # KCNQ4.load_state_dict(torch.load(f=MODEL_SAVE_PATH))
    # KCNQ4.to(device)

    # #print(f"Loss values: {loss_values[:5]}")

    # beta = 0.5 # interpolation parameter    
    # params1 = KCNQ1.named_parameters()
    # params2 = KCNQ2.named_parameters()
    # params3 = KCNQ3.named_parameters()
    # params4 = KCNQ4.named_parameters()

    # dict_params2 = dict(params2)
    # dict_params4 = dict(params4)

    # for name1, param1 in params1:
    #   if name1 in dict_params2:
    #       dict_params2[name1].data.copy_(beta*param1.data + (1-beta)*dict_params2[name1].data)

    # print(params1)

    # for name1, param1 in params3:
    #   if name1 in dict_params4:
    #       dict_params4[name1].data.copy_(beta*param1.data + (1-beta)*dict_params4[name1].data)

    # for name1, param1 in dict_params2:
    #   if name1 in dict_params4:
    #       dict_params4[name1].data.copy_(beta*param1.data + (1-beta)*dict_params4[name1].data)

    # KCNQ5.load_state_dict(dict_params4)

RuntimeError: ignored

In [ ]:
# with torch.inference_mode():
#   output_preds_new = KCNQ()